In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold, GroupKFold, StratifiedKFold
import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import seaborn as sns
from tqdm import tqdm
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.preprocessing import LabelEncoder
import time
from catboost import CatBoostRegressor

DATA_PATH = "../input/ashrae-energy-prediction/"
LEAKDATA_PATH = "../input/leakdata/"



In [3]:
train = pd.read_pickle("train.pkl", compression = 'zip')
test = pd.read_pickle("test.pkl", compression = 'zip')

In [4]:
remove_features = ['year','timestamp','meter_reading', "precip_depth_1_hr", "sea_level_pressure", "wind_direction", "wind_speed"]
features = []
for col in train.columns:
    if col not in remove_features:
        features.append(col)
#for col in ['building_id', 'site_id','meter','primary_use','square_feet','year_built','floor_count']:
#    try:features.remove(col)
#    except:None

In [5]:


categorical_feats = []

## Train

In [ ]:
##building_id wise
import time
t1 = time.time()
num_folds = 3
#folds = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)
folds = KFold(n_splits = num_folds, shuffle = False, random_state = 42)
#train.dropna(subset=['meter_reading'], inplace=True)
t = []
tr_oof = []
a = 0
for building_id in train['building_id'].unique():
    a += 1
    print(a)
    for meter in train[train['building_id'] == building_id]['meter'].unique():
        train_small = train[(train['building_id'] == building_id) & (train['meter'] == meter)]
        test_small = test[(test['building_id'] == building_id) & (test['meter'] == meter)]
        oof = np.zeros(len(train_small))
        predictions = np.zeros(len(test_small))
        target = np.log1p(train_small["meter_reading"])
        for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_small, train_small['month'])):
            tr_x, tr_y = train_small.iloc[trn_idx][features], target.iloc[trn_idx]
            vl_x, vl_y = train_small.iloc[val_idx][features], target.iloc[val_idx]

#             clf = lgb.LGBMRegressor(n_estimators=6000,
#                                         learning_rate=0.2,
#                                         feature_fraction=0.9,
#                                         subsample=0.2,  
#                                         subsample_freq=1,
#                                         num_leaves=20,
#                                         metric='rmse')
            clf = CatBoostRegressor(
                    iterations=5000, learning_rate=0.03,
                    depth=6, l2_leaf_reg=3,
                    loss_function='RMSE',
                    eval_metric='RMSE',
                    random_seed=42,
                    task_type = "GPU",
                    devices='0:1')
            #if all(tr_y)!=0:
            if len(np.unique(tr_y))!=1:
                clf.fit(tr_x, tr_y,eval_set=[(vl_x, vl_y)],early_stopping_rounds=100,verbose=2000,use_best_model=True)
                oof[val_idx] = clf.predict(train_small.iloc[val_idx][features])
                predictions += np.expm1(clf.predict(test_small[features]))
            else:
                v1 = np.unique(tr_y)[0]
                oof[val_idx] = np.array([v1]*len(val_idx))  
                predictions += np.expm1(np.array([v1]*len(test_small)))          
        test_small['meter_reading_building_id'] = predictions / num_folds
        test_small['meter_reading_building_id'] = test_small['meter_reading_building_id'].clip(train_small["meter_reading"].min(),train_small["meter_reading"].max())
        train_small['meter_reading_p'] = oof
        t.append(test_small[['meter_reading_building_id']])
        tr_oof.append(train_small[['meter_reading_p']])
t = pd.concat(t)
t = t.sort_index()
oof = pd.concat(tr_oof)
oof = oof.sort_index()
#print(np.sqrt(mean_squared_error(target, oof['meter_reading_p'])))
try:del test['meter_reading_building_id']
except:None
test = test.merge(t, left_index=True, right_index=True, how='left')
target = np.log1p(train["meter_reading"])
print(np.sqrt(mean_squared_error(target, oof['meter_reading_p'])))
oof2 = oof.copy()

t2 = time.time()
print('elasped time:',t2-t1)
#0.8102309160560927 lgb
#0.8999277997669956 cb

1
0:	learn: 0.3322354	test: 0.2639129	best: 0.2639129 (0)	total: 79.7ms	remaining: 6m 38s
bestTest = 0.2023095418
bestIteration = 41
Shrink model to first 42 iterations.
0:	learn: 0.3173021	test: 0.3039293	best: 0.3039293 (0)	total: 45.7ms	remaining: 3m 48s
bestTest = 0.2228070578
bestIteration = 179
Shrink model to first 180 iterations.
0:	learn: 0.2825011	test: 0.3659288	best: 0.3659288 (0)	total: 46.4ms	remaining: 3m 51s
bestTest = 0.245626413
bestIteration = 211
Shrink model to first 212 iterations.
2
0:	learn: 0.9860082	test: 1.8660834	best: 1.8660834 (0)	total: 64.1ms	remaining: 5m 20s
bestTest = 1.036709942
bestIteration = 502
Shrink model to first 503 iterations.
0:	learn: 1.2894206	test: 1.2757664	best: 1.2757664 (0)	total: 41.5ms	remaining: 3m 27s
bestTest = 1.275766384
bestIteration = 0
Shrink model to first 1 iterations.
0:	learn: 1.2802349	test: 1.2341349	best: 1.2341349 (0)	total: 45.8ms	remaining: 3m 48s
bestTest = 0.9312692563
bestIteration = 348
Shrink model to first 3

bestTest = 0.07295825168
bestIteration = 530
Shrink model to first 531 iterations.
0:	learn: 0.1363070	test: 0.1276886	best: 0.1276886 (0)	total: 40ms	remaining: 3m 20s
bestTest = 0.0589183329
bestIteration = 195
Shrink model to first 196 iterations.
0:	learn: 0.1343425	test: 0.1334323	best: 0.1334323 (0)	total: 38.6ms	remaining: 3m 13s
bestTest = 0.06271399151
bestIteration = 140
Shrink model to first 141 iterations.
13
0:	learn: 0.2617679	test: 0.2344615	best: 0.2344615 (0)	total: 41.5ms	remaining: 3m 27s
bestTest = 0.1609079929
bestIteration = 85
Shrink model to first 86 iterations.
0:	learn: 0.2275846	test: 0.2994290	best: 0.2994290 (0)	total: 39.8ms	remaining: 3m 19s
bestTest = 0.1409798126
bestIteration = 519
Shrink model to first 520 iterations.
0:	learn: 0.2499547	test: 0.2501741	best: 0.2501741 (0)	total: 34.3ms	remaining: 2m 51s
bestTest = 0.1572015523
bestIteration = 127
Shrink model to first 128 iterations.
0:	learn: 1.2487302	test: 1.0889483	best: 1.0889483 (0)	total: 37.9

Shrink model to first 623 iterations.
0:	learn: 0.2722291	test: 0.3368308	best: 0.3368308 (0)	total: 34ms	remaining: 2m 50s
bestTest = 0.1760814482
bestIteration = 98
Shrink model to first 99 iterations.
0:	learn: 0.2865333	test: 0.2872405	best: 0.2872405 (0)	total: 36.7ms	remaining: 3m 3s
bestTest = 0.2816340888
bestIteration = 7
Shrink model to first 8 iterations.
26
0:	learn: 0.3098534	test: 0.3221425	best: 0.3221425 (0)	total: 35.5ms	remaining: 2m 57s
bestTest = 0.1366378604
bestIteration = 475
Shrink model to first 476 iterations.
0:	learn: 0.3263391	test: 0.2885169	best: 0.2885169 (0)	total: 37.8ms	remaining: 3m 8s
bestTest = 0.1103678126
bestIteration = 120
Shrink model to first 121 iterations.
0:	learn: 0.2988991	test: 0.3421844	best: 0.3421844 (0)	total: 40.3ms	remaining: 3m 21s
bestTest = 0.2074918021
bestIteration = 133
Shrink model to first 134 iterations.
27
0:	learn: 0.4528568	test: 0.7506512	best: 0.7506512 (0)	total: 34.5ms	remaining: 2m 52s
bestTest = 0.5564314922
best

bestTest = 0.2340808461
bestIteration = 163
Shrink model to first 164 iterations.
0:	learn: 0.2864196	test: 0.3650613	best: 0.3650613 (0)	total: 44.4ms	remaining: 3m 42s
bestTest = 0.2414101711
bestIteration = 278
Shrink model to first 279 iterations.
40
0:	learn: 1.2158772	test: 1.4442036	best: 1.4442036 (0)	total: 36.4ms	remaining: 3m 1s
bestTest = 1.444203602
bestIteration = 0
Shrink model to first 1 iterations.
0:	learn: 1.2602495	test: 1.5588120	best: 1.5588120 (0)	total: 39.9ms	remaining: 3m 19s
bestTest = 0.1078157861
bestIteration = 281
Shrink model to first 282 iterations.
0:	learn: 1.1452542	test: 1.6422010	best: 1.6422010 (0)	total: 39.1ms	remaining: 3m 15s
bestTest = 1.642200973
bestIteration = 0
Shrink model to first 1 iterations.
0:	learn: 1.0896291	test: 1.7524735	best: 1.7524735 (0)	total: 33.9ms	remaining: 2m 49s
bestTest = 1.436000616
bestIteration = 311
Shrink model to first 312 iterations.
0:	learn: 1.2939922	test: 1.3403608	best: 1.3403608 (0)	total: 33.4ms	remaini

Shrink model to first 170 iterations.
0:	learn: 0.5012966	test: 0.4705282	best: 0.4705282 (0)	total: 38.8ms	remaining: 3m 13s
bestTest = 0.190871182
bestIteration = 145
Shrink model to first 146 iterations.
54
0:	learn: 0.4712925	test: 0.4661216	best: 0.4661216 (0)	total: 36.2ms	remaining: 3m
bestTest = 0.2493016542
bestIteration = 373
Shrink model to first 374 iterations.
0:	learn: 0.4146388	test: 0.5816452	best: 0.5816452 (0)	total: 36.3ms	remaining: 3m 1s
bestTest = 0.2146603176
bestIteration = 770
Shrink model to first 771 iterations.
0:	learn: 0.4857095	test: 0.4226364	best: 0.4226364 (0)	total: 42.3ms	remaining: 3m 31s
bestTest = 0.1877541513
bestIteration = 265
Shrink model to first 266 iterations.
55
0:	learn: 0.3563678	test: 0.4514253	best: 0.4514253 (0)	total: 38.4ms	remaining: 3m 12s
bestTest = 0.1761644535
bestIteration = 203
Shrink model to first 204 iterations.
0:	learn: 0.2972164	test: 0.5454635	best: 0.5454635 (0)	total: 39ms	remaining: 3m 14s
bestTest = 0.1735051611
be

...

Shrink model to first 271 iterations.
0:	learn: 0.6239832	test: 1.5065320	best: 1.5065320 (0)	total: 29ms	remaining: 2m 24s
bestTest = 0.6293373109
bestIteration = 262
Shrink model to first 263 iterations.
0:	learn: 0.9678830	test: 0.7477807	best: 0.7477807 (0)	total: 27.6ms	remaining: 2m 17s
bestTest = 0.6806008539
bestIteration = 44
Shrink model to first 45 iterations.
865
0:	learn: 0.3789973	test: 0.4049216	best: 0.4049216 (0)	total: 32.7ms	remaining: 2m 43s
bestTest = 0.06434298713
bestIteration = 300
Shrink model to first 301 iterations.
0:	learn: 0.2782166	test: 0.5607334	best: 0.5607334 (0)	total: 33.1ms	remaining: 2m 45s
bestTest = 0.191764027
bestIteration = 709
Shrink model to first 710 iterations.
0:	learn: 0.3786248	test: 0.3471267	best: 0.3471267 (0)	total: 29.6ms	remaining: 2m 28s
bestTest = 0.2064864756
bestIteration = 246
Shrink model to first 247 iterations.
0:	learn: 2.5983803	test: 2.7150762	best: 2.7150762 (0)	total: 31.9ms	remaining: 2m 39s
bestTest = 0.2235424061


Shrink model to first 309 iterations.
0:	learn: 0.2815159	test: 0.2790417	best: 0.2790417 (0)	total: 32.4ms	remaining: 2m 42s
bestTest = 0.1551279708
bestIteration = 82
Shrink model to first 83 iterations.
0:	learn: 0.2845670	test: 0.2704503	best: 0.2704503 (0)	total: 33.2ms	remaining: 2m 46s
bestTest = 0.1590521585
bestIteration = 100
Shrink model to first 101 iterations.
877
0:	learn: 0.1060308	test: 0.1106839	best: 0.1106839 (0)	total: 28.1ms	remaining: 2m 20s
bestTest = 0.07158815566
bestIteration = 322
Shrink model to first 323 iterations.
0:	learn: 0.1121095	test: 0.0981090	best: 0.0981090 (0)	total: 30.5ms	remaining: 2m 32s
bestTest = 0.06245829801
bestIteration = 167
Shrink model to first 168 iterations.
0:	learn: 0.1013624	test: 0.1221483	best: 0.1221483 (0)	total: 30.1ms	remaining: 2m 30s
bestTest = 0.0793657078
bestIteration = 290
Shrink model to first 291 iterations.
878
0:	learn: 0.3195514	test: 0.3371506	best: 0.3371506 (0)	total: 26.8ms	remaining: 2m 14s
bestTest = 0.231

Shrink model to first 103 iterations.
0:	learn: 0.2422503	test: 0.2264884	best: 0.2264884 (0)	total: 31.3ms	remaining: 2m 36s
bestTest = 0.1073339156
bestIteration = 93
Shrink model to first 94 iterations.
0:	learn: 0.2324535	test: 0.2473840	best: 0.2473840 (0)	total: 33.4ms	remaining: 2m 47s
bestTest = 0.1545867381
bestIteration = 104
Shrink model to first 105 iterations.
893
0:	learn: 0.3629534	test: 0.3531106	best: 0.3531106 (0)	total: 28.7ms	remaining: 2m 23s
bestTest = 0.1651269306
bestIteration = 114
Shrink model to first 115 iterations.
0:	learn: 0.3557382	test: 0.3689502	best: 0.3689502 (0)	total: 35.2ms	remaining: 2m 55s
bestTest = 0.162445078
bestIteration = 172
Shrink model to first 173 iterations.
0:	learn: 0.3604566	test: 0.3581755	best: 0.3581755 (0)	total: 31.5ms	remaining: 2m 37s
bestTest = 0.1701396341
bestIteration = 125
Shrink model to first 126 iterations.
894
0:	learn: 0.4757969	test: 0.4942239	best: 0.4942239 (0)	total: 31.2ms	remaining: 2m 35s
bestTest = 0.418196

Shrink model to first 1009 iterations.
0:	learn: 0.5336107	test: 0.7221179	best: 0.7221179 (0)	total: 31ms	remaining: 2m 35s
bestTest = 0.2884920864
bestIteration = 86
Shrink model to first 87 iterations.
0:	learn: 0.5778370	test: 0.5483922	best: 0.5483922 (0)	total: 34.2ms	remaining: 2m 50s
bestTest = 0.5335706631
bestIteration = 172
Shrink model to first 173 iterations.
0:	learn: 0.0672347	test: 0.0916894	best: 0.0916894 (0)	total: 35.5ms	remaining: 2m 57s
bestTest = 0.08177687948
bestIteration = 48
Shrink model to first 49 iterations.
0:	learn: 0.0773037	test: 0.0782170	best: 0.0782170 (0)	total: 33ms	remaining: 2m 44s
bestTest = 0.06213195521
bestIteration = 44
Shrink model to first 45 iterations.
0:	learn: 0.0811784	test: 0.0695152	best: 0.0695152 (0)	total: 30.7ms	remaining: 2m 33s
bestTest = 0.05978123334
bestIteration = 32
Shrink model to first 33 iterations.
907
0:	learn: 0.4251601	test: 0.4789371	best: 0.4789371 (0)	total: 27.4ms	remaining: 2m 17s
bestTest = 0.2040212261
best

bestTest = 0.6231789998
bestIteration = 305
Shrink model to first 306 iterations.
0:	learn: 2.2328356	test: 2.0236673	best: 2.0236673 (0)	total: 32.4ms	remaining: 2m 41s
bestTest = 0.870551134
bestIteration = 253
Shrink model to first 254 iterations.
0:	learn: 2.5871408	test: 2.8117508	best: 2.8117508 (0)	total: 26.2ms	remaining: 2m 11s
bestTest = 1.96423989
bestIteration = 717
Shrink model to first 718 iterations.
0:	learn: 2.6506440	test: 2.3155262	best: 2.3155262 (0)	total: 29.5ms	remaining: 2m 27s
bestTest = 1.664174636
bestIteration = 35
Shrink model to first 36 iterations.
0:	learn: 2.0957969	test: 3.3174857	best: 3.3174857 (0)	total: 28.7ms	remaining: 2m 23s
bestTest = 2.307213366
bestIteration = 81
Shrink model to first 82 iterations.
914
0:	learn: 0.2698574	test: 0.3433777	best: 0.3433777 (0)	total: 27.6ms	remaining: 2m 17s
bestTest = 0.2054251426
bestIteration = 86
Shrink model to first 87 iterations.
0:	learn: 0.2985097	test: 0.2863043	best: 0.2863043 (0)	total: 29.8ms	remai

In [ ]:
##site wise
import time
t1 = time.time()
num_folds = 3
#folds = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)
folds = KFold(n_splits = num_folds, shuffle = False, random_state = 42)
#train.dropna(subset=['meter_reading'], inplace=True)
t = []
tr_oof = []
a = 0
for site_id in train['site_id'].unique():
    a += 1
    print(a)
    for meter in train[train['site_id'] == site_id]['meter'].unique():
        train_small = train[(train['site_id'] == site_id) & (train['meter'] == meter)]
        test_small = test[(test['site_id'] == site_id) & (test['meter'] == meter)]
        oof = np.zeros(len(train_small))
        predictions = np.zeros(len(test_small))
        target = np.log1p(train_small["meter_reading"])
        for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_small, train_small['month'])):
            tr_x, tr_y = train_small.iloc[trn_idx][features], target.iloc[trn_idx]
            vl_x, vl_y = train_small.iloc[val_idx][features], target.iloc[val_idx]

#             clf = get_clf(site_id)
            clf = CatBoostRegressor(
                    iterations=5000, learning_rate=0.03,
                    depth=6, l2_leaf_reg=3,
                    loss_function='RMSE',
                    eval_metric='RMSE',
                    random_seed=42,
                    task_type = "GPU",
                    devices='2:3')
            clf.fit(tr_x, tr_y,eval_set=[(vl_x, vl_y)],early_stopping_rounds=100,verbose=2000,use_best_model=True)
            oof[val_idx] = clf.predict(train_small.iloc[val_idx][features])
            predictions += np.expm1(clf.predict(test_small[features]))
        test_small['meter_reading_site_id'] = predictions / num_folds
        test_small['meter_reading_site_id'] = test_small['meter_reading_site_id'].clip(train_small["meter_reading"].min(),train_small["meter_reading"].max())
        train_small['meter_reading_p'] = oof
        t.append(test_small[['meter_reading_site_id']])
        tr_oof.append(train_small[['meter_reading_p']])
        

t = pd.concat(t)
t = t.sort_index()
oof = pd.concat(tr_oof)
oof = oof.sort_index()
# print(np.sqrt(mean_squared_error(target, oof['meter_reading_p'])))

try:del test['meter_reading_site_id']
except:None
test = test.merge(t, left_index=True, right_index=True, how='left')
target = np.log1p(train["meter_reading"])
print(np.sqrt(mean_squared_error(target, oof['meter_reading_p'])))
oof1 = oof.copy()

t2 = time.time()
print('elasped time:',t2-t1)

#0.8864115837463974 lgb

#0.9193790471358488 cat

In [ ]:
##primary_use wise

import time
t1 = time.time()
num_folds = 4
#folds = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)
folds = KFold(n_splits = num_folds, shuffle = False, random_state = 42)
#train.dropna(subset=['meter_reading'], inplace=True)
t = []
tr_oof = []
a = 0
for primary_use in train['primary_use'].unique():
    a += 1
    print(a)
    for meter in train[train['primary_use'] == primary_use]['meter'].unique():
        #print('primary_use-meter:',primary_use,'-',meter)
        train_small = train[(train['primary_use'] == primary_use) & (train['meter'] == meter)]
        test_small = test[(test['primary_use'] == primary_use) & (test['meter'] == meter)]
        oof = np.zeros(len(train_small))
        predictions = np.zeros(len(test_small))
        target = np.log1p(train_small["meter_reading"])
        for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_small, train_small['month'])):
            #print('primary_use-meter-fold:',primary_use,'-',meter,'-',fold_)
            tr_x, tr_y = train_small.iloc[trn_idx][features], target.iloc[trn_idx]
            vl_x, vl_y = train_small.iloc[val_idx][features], target.iloc[val_idx]
            clf = CatBoostRegressor(
                    iterations=5000, learning_rate=0.03,
                    depth=6, l2_leaf_reg=3,
                    loss_function='RMSE',
                    eval_metric='RMSE',
                    random_seed=42,
                    task_type = "GPU",
                    devices='2:3')
            clf.fit(tr_x, tr_y,eval_set=[(vl_x, vl_y)],early_stopping_rounds=100,verbose=2000,use_best_model=True)
            oof[val_idx] = clf.predict(train_small.iloc[val_idx][features])
            predictions += np.expm1(clf.predict(test_small[features]))
        test_small['meter_reading_primary_use'] = predictions / num_folds
        test_small['meter_reading_primary_use'] = test_small['meter_reading_primary_use'].clip(train_small["meter_reading"].min(),train_small["meter_reading"].max())
        train_small['meter_reading_p'] = oof
        t.append(test_small[['meter_reading_primary_use']])
        tr_oof.append(train_small[['meter_reading_p']])
t = pd.concat(t)
t = t.sort_index()
oof = pd.concat(tr_oof)
oof = oof.sort_index()
#print(np.sqrt(mean_squared_error(target, oof['meter_reading_p'])))
try:del test['meter_reading_primary_use']
except:None
test = test.merge(t, left_index=True, right_index=True, how='left')
target = np.log1p(train["meter_reading"])
print(np.sqrt(mean_squared_error(target, oof['meter_reading_p'])))
t2 = time.time()
print('elasped time:',t2-t1)
oof3 = oof.copy()
#0.8636186523644124 lgb
#0.8692092058911276 cb

In [ ]:
# oof_save = oof1.copy()
# oof_save['meter_reading_site_id'] = oof1['meter_reading_p']
# oof_save['meter_reading_primary_use'] = oof3['meter_reading_p']
# oof_save['meter_reading_building_id'] = oof2['meter_reading_p']
# test_save = test[['meter_reading_site_id','meter_reading_primary_use','meter_reading_building_id']]
# test_save.to_csv("test_cb_save.csv")
# oof_save.to_csv("oof_cb_save.csv")


In [ ]:
# print(np.sqrt(mean_squared_error(target, oof_save['meter_reading_site_id']*0.1+oof_save['meter_reading_building_id']*0.3+oof_save['meter_reading_primary_use']*0.6)))



In [ ]:
# print(np.sqrt(mean_squared_error(target, oof1['meter_reading_p']*0.1+oof2['meter_reading_p']*0.3+oof3['meter_reading_p']*0.6)))



In [ ]:
sub = pd.read_csv(DATA_PATH+"sample_submission.csv")
#sub["meter_reading"] = (test['meter_reading_site_id'] * 0.6 + test['meter_reading_primary_use'] * 0.2 + test['meter_reading_building_id'] * 0.2)
#sub.to_csv("best_more_deleting.gz",  index = False, compression= 'gzip')

In [ ]:
test["meter_reading"] = test['meter_reading_site_id'] * 0.1 + test['meter_reading_primary_use'] * 0.6 + test['meter_reading_building_id'] * 0.3

In [ ]:
###Postprocess 
test['meter_reading'] = [meter_r * 0.85 if site not in [0,1,2,3,4,15] else meter_r for meter_r,site in zip(test['meter_reading'], test['site_id'])]

In [ ]:
leak_df = pd.read_csv(LEAKDATA_PATH+'leak.csv')
leak_df['timestamp'] = pd.to_datetime(leak_df["timestamp"])
leak_df.dropna(0, inplace=True)
leak_df = leak_df[(leak_df.timestamp.dt.year > 2016) & (leak_df.timestamp.dt.year < 2019)]
leak_df.loc[leak_df.meter_reading < 0, 'meter_reading'] = 0 # remove large negative values
leak_df = leak_df[leak_df.building_id!=245]
leak_df['meter_reading_sc'] = leak_df['meter_reading'] 
del leak_df['meter_reading'] 
test = test.merge(leak_df, on=['building_id','meter','timestamp'], how='left')

In [ ]:
test['meter_reading'] = [a if (a >= 0 and site != 4) or (a < 0 and site != 4) else b for a,b,site in zip(test['meter_reading_sc'], test['meter_reading'], test['site_id'])]

In [ ]:
sub["meter_reading"] = test['meter_reading']
sub.to_csv("../sub/cb_3wise.csv",  index = False,float_format="%.4f")

In [ ]:
sub.head()